In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import random
import numpy as np
import torch
import cv2
import timm
import pandas as pd
import torchvision.transforms as transform
from sklearn import model_selection
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
from timm.models import create_model

from dataset import CustomDataset, MultiviewImgDataset, SingleimgDataset
from sklearn.metrics import classification_report
import models.create_models as create
# from ml_decoder import MLDecoder

/home/cyn/anaconda3/envs/cyn1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# DATA_PATH = "/disk1/chenyangneng/data/MFIDDR"
# TEST_PATH = "/disk1/chenyangneng/data/MFIDDR/test/"
# test_csv_path = os.path.join(DATA_PATH, 'test_fourpic_label.csv')
# test_df = pd.read_csv(test_csv_path)
# # 创建数据集实例
# test_dataset = MultiviewImgDataset(TEST_PATH, test_df)



In [3]:
from typing import Optional

import torch
from torch import nn, Tensor
from torch.nn.modules.transformer import _get_activation_fn


def add_ml_decoder_head(
    model,
    num_classes=-1,
    num_of_groups=-1,
    decoder_embedding=768,
    zsl=0,
    num_layers_decoder=1,  # 修改
):
    if num_classes == -1:
        num_classes = model.num_classes
    num_features = model.num_features
    if hasattr(model, "global_pool") and hasattr(model, "fc"):  # resnet50
        model.global_pool = nn.Identity()
        del model.fc
        model.fc = MLDecoder(
            num_classes=num_classes,
            initial_num_features=num_features,
            num_of_groups=num_of_groups,
            decoder_embedding=decoder_embedding,
            zsl=zsl,
            num_layers_decoder=num_layers_decoder,
        )
    elif hasattr(model, "head"):  # tresnet
        if hasattr(model, "global_pool"):
            model.global_pool = nn.Identity()
        del model.head
        model.head = MLDecoder(
            num_classes=num_classes,
            initial_num_features=num_features,
            num_of_groups=num_of_groups,
            decoder_embedding=decoder_embedding,
            zsl=zsl,
        )
    else:
        print("model is not suited for ml-decoder")
        exit(-1)

    return model


class TransformerDecoderLayerOptimal(nn.Module):
    def __init__(
        self,
        d_model,
        nhead=8,
        dim_feedforward=2048,
        dropout=0.1,
        activation="relu",
        layer_norm_eps=1e-5,
    ) -> None:
        super(TransformerDecoderLayerOptimal, self).__init__()
        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.dropout = nn.Dropout(dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)

        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm3 = nn.LayerNorm(d_model, eps=layer_norm_eps)

        self.activation = _get_activation_fn(activation)

    def __setstate__(self, state):
        if "activation" not in state:
            state["activation"] = torch.nn.functional.relu
        super(TransformerDecoderLayerOptimal, self).__setstate__(state)

    def forward(
        self,
        tgt: Tensor,
        memory: Tensor,
        tgt_mask: Optional[Tensor] = None,
        memory_mask: Optional[Tensor] = None,
        tgt_key_padding_mask: Optional[Tensor] = None,
        memory_key_padding_mask: Optional[Tensor] = None,
    ) -> Tensor:
        tgt = tgt + self.dropout1(tgt)
        tgt = self.norm1(tgt)
        print("-----------",tgt.shape,memory.shape)
        tgt2 = self.multihead_attn(tgt, memory, memory)[0]
        print(tgt2.shape)
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt


# @torch.jit.script
# class ExtrapClasses(object):
#     def __init__(self, num_queries: int, group_size: int):
#         self.num_queries = num_queries
#         self.group_size = group_size
#
#     def __call__(self, h: torch.Tensor, class_embed_w: torch.Tensor, class_embed_b: torch.Tensor, out_extrap:
#     torch.Tensor):
#         # h = h.unsqueeze(-1).expand(-1, -1, -1, self.group_size)
#         h = h[..., None].repeat(1, 1, 1, self.group_size) # torch.Size([bs, 5, 768, groups])
#         w = class_embed_w.view((self.num_queries, h.shape[2], self.group_size))
#         out = (h * w).sum(dim=2) + class_embed_b
#         out = out.view((h.shape[0], self.group_size * self.num_queries))
#         return out


# @torch.jit.script
class GroupFC(object):
    def __init__(self, embed_len_decoder: int):
        self.embed_len_decoder = embed_len_decoder

    def __call__(
        self, h: torch.Tensor, duplicate_pooling: torch.Tensor, out_extrap: torch.Tensor
    ):
        for i in range(h.shape[1]):
            h_i = h[:, i, :]
            if len(duplicate_pooling.shape) == 3:
                w_i = duplicate_pooling[i, :, :]
            else:
                w_i = duplicate_pooling
            out_extrap[:, i, :] = torch.matmul(h_i, w_i)


class MLDecoder(nn.Module):
    def __init__(
        self,
        num_classes,
        num_of_groups=-1,
        decoder_embedding=768,# 3072
        initial_num_features=2048,# 3072
        zsl=0,
        num_layers_decoder=1,  # 修改
    ):
        super(MLDecoder, self).__init__()
        embed_len_decoder = 100 if num_of_groups < 0 else num_of_groups
        if embed_len_decoder > num_classes:
            embed_len_decoder = num_classes

        # switching to 768 initial embeddings
        decoder_embedding = 768 if decoder_embedding < 0 else decoder_embedding
        embed_standart = nn.Linear(initial_num_features, decoder_embedding)

        # non-learnable queries
        if not zsl:
            query_embed = nn.Embedding(embed_len_decoder, decoder_embedding)
            query_embed.requires_grad_(False)
        else:
            query_embed = None

        # decoder
        decoder_dropout = 0.1
        # num_layers_decoder = 5
        dim_feedforward = 2048
        layer_decode = TransformerDecoderLayerOptimal(
            d_model=decoder_embedding,
            dim_feedforward=dim_feedforward,
            dropout=decoder_dropout,
        )
        self.decoder = nn.TransformerDecoder(
            layer_decode, num_layers=num_layers_decoder
        )
        # self.decoder = layer_decode  # 修改
        self.decoder.embed_standart = embed_standart
        self.decoder.query_embed = query_embed
        self.zsl = zsl

        if self.zsl:
            if decoder_embedding != 300:
                self.wordvec_proj = nn.Linear(300, decoder_embedding)
            else:
                self.wordvec_proj = nn.Identity()
            self.decoder.duplicate_pooling = torch.nn.Parameter(
                torch.Tensor(decoder_embedding, 1)
            )
            self.decoder.duplicate_pooling_bias = torch.nn.Parameter(torch.Tensor(1))
            self.decoder.duplicate_factor = 1
        else:
            # group fully-connected
            self.decoder.num_classes = num_classes
            self.decoder.duplicate_factor = int(num_classes / embed_len_decoder + 0.999)
            self.decoder.duplicate_pooling = torch.nn.Parameter(
                torch.Tensor(
                    embed_len_decoder, decoder_embedding, self.decoder.duplicate_factor
                )
            )
            self.decoder.duplicate_pooling_bias = torch.nn.Parameter(
                torch.Tensor(num_classes)
            )
        torch.nn.init.xavier_normal_(self.decoder.duplicate_pooling)
        torch.nn.init.constant_(self.decoder.duplicate_pooling_bias, 0)
        self.decoder.group_fc = GroupFC(embed_len_decoder)
        self.train_wordvecs = None
        self.test_wordvecs = None

    def forward(self, x):
        if len(x.shape) == 4:  # [bs,2048, 7,7] [bs,3072,17,17]
            embedding_spatial = x.flatten(2).transpose(1, 2) #  [bs,289,3072]
        else:  # [bs, 197,468]
            embedding_spatial = x
        embedding_spatial_786 = self.decoder.embed_standart(embedding_spatial)# linear(3072,3072)
        embedding_spatial_786 = torch.nn.functional.relu(
            embedding_spatial_786, inplace=True
        )

        bs = embedding_spatial_786.shape[0]
        if self.zsl:
            query_embed = torch.nn.functional.relu(
                self.wordvec_proj(self.decoder.query_embed)
            )
        else:
            query_embed = self.decoder.query_embed.weight
        # tgt = query_embed.unsqueeze(1).repeat(1, bs, 1)
        # query_embed [100,3072]
        tgt = query_embed.unsqueeze(1).expand(
            -1, bs, -1
        )  # no allocation of memory with expand 
        # tgt [100,bs,3072]
        # embedding_spatial_786 [bs,289,3072]
        print("tgt",tgt.shape)
        print("embedding_spatial_786",embedding_spatial_786.shape)
        h = self.decoder(
            tgt, embedding_spatial_786.transpose(0, 1)
        )  # [embed_len_decoder, batch, 768]
        h = h.transpose(0, 1)
        print("h",h.shape)
        out_extrap = torch.zeros(
            h.shape[0],
            h.shape[1],
            self.decoder.duplicate_factor,
            device=h.device,
            dtype=h.dtype,
        )
        self.decoder.group_fc(h, self.decoder.duplicate_pooling, out_extrap)
        print("s-----",out_extrap.shape)
        if not self.zsl:
            h_out = out_extrap.flatten(1)[:, : self.decoder.num_classes]
        else:
            h_out = out_extrap.flatten(1)
        h_out += self.decoder.duplicate_pooling_bias
        logits = h_out
        return logits


In [4]:
head=MLDecoder(num_classes=5,decoder_embedding=3072,initial_num_features=3072,num_layers_decoder=8,num_of_groups=5)

In [5]:
h=torch.zeros(8,3072,17,17)

In [6]:
head(h)

tgt torch.Size([5, 8, 3072])
embedding_spatial_786 torch.Size([8, 289, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
----------- torch.Size([5, 8, 3072]) torch.Size([289, 8, 3072])
torch.Size([5, 8, 3072])
h torch.Size([8, 5, 3072])
s----- torch.Size([8, 5, 1])


tensor([[-1.1128, -1.6580,  1.3629,  0.6822,  0.1324],
        [-1.6582, -1.4632,  0.4433,  1.6761, -0.6224],
        [-2.0831, -0.9010,  1.5078, -0.1696,  0.0536],
        [-0.8993, -1.1478,  1.3426,  1.3557, -0.0658],
        [-1.7047,  0.1282,  1.8003,  0.8549,  0.5491],
        [-1.9468, -0.6957,  0.5339,  1.3930, -0.0485],
        [-0.5730, -0.5004,  2.5687,  1.4899,  0.1522],
        [-1.7733, -1.3661,  1.7731,  1.8172,  0.5989]],
       grad_fn=<AsStridedBackward0>)